In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from scipy.stats import lognorm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import SequentialFeatureSelector
%matplotlib inline
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df = pd.read_csv('train-2.csv', converters={'features': eval})
df = df.rename(str.lower, axis='columns',)
df = df.set_index('id')
display(df)

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target
id,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25582,1.0,1,14fdc4b01ae44b025f6c4d28c9097e5f,2016-06-16 02:12:57,Newly renovated bedroom apartment located off ...,29th St,[No pets],40.7653,7166867,-73.9248,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7166867_625fde3...,1950,30-95 29th St,low
50013,1.0,0,9b6cf886379a2511f8c633c84028efe7,2016-05-10 03:17:32,All apartments are newly renovated featuring: ...,E 1st St,"[Pre-War, Laundry in Unit, Dishwasher, Hardwoo...",40.7239,6989251,-73.9901,5599e962719af3ccc2976855c2d5893c,['https://photos.renthop.com/2/6989251_42a4884...,2600,39 E 1st St,medium
111475,2.0,2,0,2016-04-21 03:29:35,"<![CDATA[2 bedrooms, 5110, Astoria / Long Isla...",50th Avenue,[],40.7437,6905568,-73.9585,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/6905568_01602e3...,5110,2-01 50th Avenue,low


In [4]:
y = df['target']
display(y)
X = df.drop(['target'], axis=1)
display(X.head())


id
57094     medium
33389     medium
60458        low
53048        low
592          low
           ...  
25582        low
50013     medium
111475       low
71184     medium
117473    medium
Name: target, Length: 34546, dtype: object

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
id,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street


In [5]:
display(df.shape)
display(df.info())
display(df.isna().sum())

(34546, 15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34546 entries, 57094 to 117473
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bathrooms        34546 non-null  float64
 1   bedrooms         34546 non-null  int64  
 2   building_id      34546 non-null  object 
 3   created          34546 non-null  object 
 4   description      33509 non-null  object 
 5   display_address  34458 non-null  object 
 6   features         34546 non-null  object 
 7   latitude         34546 non-null  float64
 8   listing_id       34546 non-null  int64  
 9   longitude        34546 non-null  float64
 10  manager_id       34546 non-null  object 
 11  photos           34546 non-null  object 
 12  price            34546 non-null  int64  
 13  street_address   34542 non-null  object 
 14  target           34546 non-null  object 
dtypes: float64(3), int64(3), object(9)
memory usage: 4.2+ MB


None

bathrooms             0
bedrooms              0
building_id           0
created               0
description        1037
display_address      88
features              0
latitude              0
listing_id            0
longitude             0
manager_id            0
photos                0
price                 0
street_address        4
target                0
dtype: int64

In [6]:
display(df[df['target'] == 'low'].shape)

(23999, 15)

In [7]:
high_percent = len(df[df['target'] == 'high']) * 100 / len(df)
medium_percent = len(df[df['target'] == 'medium']) * 100 / len(df)
low_percent = len(df[df['target'] == 'low']) * 100 / len(df)
display(f"high: {high_percent}%")
display(f"medium: {medium_percent}%")
display(f"low: {low_percent}%")


'high: 7.778035083656574%'

'medium: 22.752272332542116%'

'low: 69.4696925838013%'

In [8]:
df['description'] = df['description'].str.lower()
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target
id,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,a fabulous 3br in midtown west! perfect apar...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,renovated kitchen and bathroom!,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,rare and best deal on the market!!!! perfect s...,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,newly renovated flex 2 apartment offers the ne...,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,low fee apartments do not come around like thi...,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low


In [9]:
def remove_trash(cur_str):
    cur_str = str(cur_str)
    nw_str = ""
    for elem in cur_str:
        if (elem >= 'a' and elem <= 'z') or elem == ' ':
            nw_str += elem

    return nw_str.split(' ')

df['description'] = df['description'].apply(remove_trash)
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target
id,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,"[a, fabulous, br, in, midtown, west, , , perfe...",HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,"[renovated, kitchen, and, bathroom]",55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,"[rare, and, best, deal, on, the, market, perfe...",W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,"[newly, renovated, flex, , apartment, offers, ...",John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,"[low, fee, apartments, do, not, come, around, ...",West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low


In [10]:
target_to_okey = {'low': 0, 'medium': 0.5, 'high' : 1}

In [11]:
description_words_weight = dict()
description_words_count = dict()
for index, row in df.iterrows():
    for word in row.description:
        if len(word) < 3:
            continue
        if word in description_words_weight:
            description_words_weight[word] += target_to_okey[row.target]
            description_words_count[word] += 1
        else:
            description_words_weight[word] = target_to_okey[row.target]
            description_words_count[word] = 1
        
for key, value in description_words_weight.items():
    description_words_weight[key] = value / (description_words_count[key])

display(description_words_weight['good'])

0.2412309728656519

In [12]:
def words_weight(x):
    res = 0.0
    for word in x:
        if word in description_words_weight:
            res += description_words_weight[word]
    if len(x) > 0:
        res /= len(x)
    return res

df['description'] = df['description'].apply(words_weight)
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target
id,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,0.153092,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,0.214983,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,0.149183,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,0.152593,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,0.143985,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low


In [83]:
str.lower("KeK")

'kek'

In [84]:
features_words_weight = dict()
features_words_count = dict()
for index, row in df.iterrows():
    for word in row.features:
        word = str.lower(word)
        if word in features_words_weight:
            features_words_weight[word] += row.temp_target
            features_words_count[word] += 1
        else:
            features_words_weight[word] = row.temp_target
            features_words_count[word] = 1
        
        
for key, value in features_words_weight.items():
    features_words_weight[key] = value / (features_words_count[key] * 100)

display(features_words_weight['no fee'])
df.head()

0.6943055699304582

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target,temp_target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,0.548578,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"[Laundry In Unit, No Fee, Elevator]",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium,92.221965,69.297433,70.320227
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,0.693230,55 River Drive South,"[Dogs Allowed, Cats Allowed, No Fee]",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium,92.221965,80.845829,74.579835
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,0.542103,W 77 Street,"[Elevator, Hardwood Floors]",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low,69.469693,66.188957,70.474858
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,0.510558,John Street,"[Swimming Pool, Doorman, Elevator, Fitness Cen...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low,69.469693,67.339708,69.829174
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,0.533959,West 16th Street,"[Laundry in Building, Laundry in Unit, Dishwas...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low,69.469693,70.878405,69.463040


In [85]:
def features_weight(x):
    res = 0.0
    for word in x:
        word = str.lower(word)
        if word in features_words_weight:
            res += features_words_weight[word]
    if len(x) > 0:
        res /= len(x)
    return res

#display(features_weight(df.iloc[0, 6]))
df['features'] = df['features'].apply(features_weight)
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,target,temp_target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,,,,,,,,,,,
57094,1.0,3,0,2016-05-19 18:06:27,0.548578,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,0.700512,40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium,92.221965,69.297433,70.320227
33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,0.693230,55 River Drive South,0.695180,40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium,92.221965,80.845829,74.579835
60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,0.542103,W 77 Street,0.701212,40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low,69.469693,66.188957,70.474858
53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,0.510558,John Street,0.704414,40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low,69.469693,67.339708,69.829174
592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,0.533959,West 16th Street,0.703955,40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low,69.469693,70.878405,69.463040


In [86]:
df.drop(['building_id', 'created', 'display_address', 'latitude', 'listing_id', 'longitude', 'manager_id', 'photos', 'street_address'], inplace = True, axis = 1)
df.head()

,bathrooms,bedrooms,description,features,price,target,temp_target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,,
57094,1.0,3,0.548578,0.700512,4495,medium,92.221965,69.297433,70.320227
33389,1.0,1,0.693230,0.695180,2570,medium,92.221965,80.845829,74.579835
60458,1.0,0,0.542103,0.701212,1795,low,69.469693,66.188957,70.474858
53048,1.0,2,0.510558,0.704414,3400,low,69.469693,67.339708,69.829174
592,1.0,3,0.533959,0.703955,5695,low,69.469693,70.878405,69.463040


In [89]:
df.drop(['temp_target'], inplace = True, axis = 1)
df.head()

,bathrooms,bedrooms,description,features,price,target,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,,
57094,1.0,3,0.548578,0.700512,4495,medium,69.297433,70.320227
33389,1.0,1,0.693230,0.695180,2570,medium,80.845829,74.579835
60458,1.0,0,0.542103,0.701212,1795,low,66.188957,70.474858
53048,1.0,2,0.510558,0.704414,3400,low,67.339708,69.829174
592,1.0,3,0.533959,0.703955,5695,low,70.878405,69.463040


In [133]:
y = df['target'].map({'low' : 1, 'medium' : 2, 'high' : 3})
X = df.drop(['target'], axis = 1)
display(X.head())
display(y)

,bathrooms,bedrooms,description,features,price,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,
57094,1.0,3,0.548578,0.700512,4495,69.297433,70.320227
33389,1.0,1,0.693230,0.695180,2570,80.845829,74.579835
60458,1.0,0,0.542103,0.701212,1795,66.188957,70.474858
53048,1.0,2,0.510558,0.704414,3400,67.339708,69.829174
592,1.0,3,0.533959,0.703955,5695,70.878405,69.463040


id
57094     2
33389     2
60458     1
53048     1
592       1
         ..
25582     1
50013     2
111475    1
71184     2
117473    2
Name: target, Length: 34546, dtype: int64

In [134]:
standart_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()
X[X.columns] = minmax_scaler.fit_transform(X[X.columns])
X.head()

,bathrooms,bedrooms,description,features,price,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,
57094,0.166667,0.375,0.681452,0.759593,0.000991,0.748695,0.759907
33389,0.166667,0.125,0.861140,0.753812,0.000562,0.875292,0.806602
60458,0.166667,0.000,0.673408,0.760353,0.000390,0.714619,0.761602
53048,0.166667,0.250,0.634223,0.763825,0.000747,0.727234,0.754524
592,0.166667,0.375,0.663292,0.763326,0.001258,0.766026,0.750510


In [135]:
X[:20]

,bathrooms,bedrooms,description,features,price,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,
57094,0.166667,0.375,0.681452,0.759593,0.000991,0.748695,0.759907
33389,0.166667,0.125,0.861140,0.753812,0.000562,0.875292,0.806602
60458,0.166667,0.000,0.673408,0.760353,0.000390,0.714619,0.761602
53048,0.166667,0.250,0.634223,0.763825,0.000747,0.727234,0.754524
592,0.166667,0.375,0.663292,0.763326,0.001258,0.766026,0.750510
31731,0.166667,0.125,0.672559,0.761708,0.000558,0.750350,0.723961
12930,0.166667,0.250,0.689411,0.756590,0.000767,0.748695,0.750583
25585,0.166667,0.125,0.598833,0.753852,0.000280,0.000000,0.472306
103527,0.166667,0.125,0.575154,0.760545,0.000413,0.875292,0.792578


In [2]:
from preprocessing import preprocess

X, y = preprocess('train-2.csv')
display(X, y)

,bathrooms,bedrooms,description,features,price,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,
57094,0.166667,0.375,0.681452,0.759593,0.000991,0.748695,0.759907
33389,0.166667,0.125,0.861140,0.753812,0.000562,0.875292,0.806602
60458,0.166667,0.000,0.673408,0.760353,0.000390,0.714619,0.761602
53048,0.166667,0.250,0.634223,0.763825,0.000747,0.727234,0.754524
592,0.166667,0.375,0.663292,0.763326,0.001258,0.766026,0.750510
...,...,...,...,...,...,...,...
25582,0.166667,0.125,0.655983,0.772366,0.000424,0.375292,0.754524
50013,0.166667,0.000,0.642653,0.759475,0.000569,0.729604,0.783151
111475,0.333333,0.250,0.653830,0.000000,0.001128,0.748695,0.754524


id
57094     2
33389     2
60458     1
53048     1
592       1
         ..
25582     1
50013     2
111475    1
71184     2
117473    2
Name: target, Length: 34546, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)

In [6]:
log_reg = LogisticRegression() 
log_reg.fit(X_train, y_train)

LogisticRegression()

In [7]:
y_pred = log_reg.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [8]:
accuracy_score(y_test, y_pred)

0.7734465457352374

In [9]:
y_dumb = np.ones(len(y_pred))
accuracy_score(y_test, y_dumb)

0.706291007333076

In [10]:
check_X, check_y = preprocess('test-2.csv')
display(check_X)
display(check_y)

,bathrooms,bedrooms,description,features,price,most_frequent_house_comment,most_frequent_rieltor_comment
id,,,,,,,
57094,0.166667,0.375,0.681452,0.759593,0.000991,0.748695,0.759907
33389,0.166667,0.125,0.861140,0.753812,0.000562,0.875292,0.806602
60458,0.166667,0.000,0.673408,0.760353,0.000390,0.714619,0.761602
53048,0.166667,0.250,0.634223,0.763825,0.000747,0.727234,0.754524
592,0.166667,0.375,0.663292,0.763326,0.001258,0.766026,0.750510
...,...,...,...,...,...,...,...
25582,0.166667,0.125,0.655983,0.772366,0.000424,0.375292,0.754524
50013,0.166667,0.000,0.642653,0.759475,0.000569,0.729604,0.783151
111475,0.333333,0.250,0.653830,0.000000,0.001128,0.748695,0.754524


id
57094     2
33389     2
60458     1
53048     1
592       1
         ..
25582     1
50013     2
111475    1
71184     2
117473    2
Name: target, Length: 34546, dtype: int64

In [ ]:
check_pred = log_reg.predict(X)

In [13]:
submission = pd.DataFrame(check_X.index)
submission

,id
0,57094
1,33389
2,60458
3,53048
4,592
...,...
34541,25582
34542,50013
34543,111475
34544,71184


In [ ]:
submission['TARGET'] = 